In [1]:
import os
import sys
# print(sys.path)
import subprocess
import shutil
import pandas as pd
import numpy as np
import calendar
import traceback
import json
from pyutils import *
from pdutils import *
from pdpltutils import *
from gputils import *
from iputils import *
import xmltodict
import re
import pytesseract

In [2]:
if sys.platform == "linux":
    DATADIR="/home/jblake1/Downloads/Network_Measurements"
else:
    DATADIR="P:\\My Drive\\CMU-LEL\\Mill19\\Images\\Coverage and Performance"
    EXPDIR=os.path.join(*[DATADIR,"2024-09-18-Mill19-PTC-Coverage","LEL-UE1"])

DIRCHECKLIST=[DATADIR,EXPDIR]
for DIR in DIRCHECKLIST:
    print(f"{DIR} exists") if os.path.isdir(DIR) else print(f"{DIR} does not exist")

P:\My Drive\CMU-LEL\Mill19\Images\Coverage and Performance exists
P:\My Drive\CMU-LEL\Mill19\Images\Coverage and Performance\2024-09-18-Mill19-PTC-Coverage\LEL-UE1 exists


In [3]:
ffnlst = walkDir(EXPDIR)
# print(ffnlst)
pngfiles = [fn for fn in ffnlst if ".png" in fn]
kmlfiles =  [fn for fn in ffnlst if ".kml" in fn ]
txtfiles =  [fn for fn in ffnlst if ".txt" in fn ]

print(f"pngfiles={len(pngfiles)} kmlfiles={len(kmlfiles)} txtfiles={len(txtfiles)}")

pngfiles=79 kmlfiles=36 txtfiles=12


In [4]:
tdfx = pd.DataFrame(pngfiles,columns=["PNGFFN"])

In [7]:
def runOCR(ffn,fn=os.path.join(EXPDIR,"tmp.txt")):
    imagetextstr = pytesseract.image_to_string(Image.open(ffn))
    print(imagetextstr)
    with open(fn,'w') as f:
        f.writelines(imagetextstr)
testffn = tdfx.PNGFFN.iloc[0]
print(testffn)
runOCR(tdfx.PNGFFN.iloc[0])

P:\My Drive\CMU-LEL\Mill19\Images\Coverage and Performance\2024-09-18-Mill19-PTC-Coverage\LEL-UE1\Screenshot_20240918-122038.png
12:20 & B 9 4a
& G-NetTrack Lite vI7.8 3

Operator: 314737
MCC: 314 MNC: 737 TAC: 1 Type: 5G NSA
eNB: 14 CID: 1 PCI: 19/ TA:
ARFCN: 637632/ BAND: N48/ BW:
F DL: 3564.5/- MHz F uL: 3564.5/- MHz
RSRP[5G/4G]: -112/ RSRQ: -11/ SNR: 11.0/
Longitude: -79.95507 Latitude: 40.415898
Speed: 3 km/h Hdg; 192° S GPS Acc: 3m
Height: 195m Altitude: 195m Ground: Om
UL: 0 kbps DL: 0 kbps

Data: No Connection IDLE

Serving time:3s
TIME EVENT, AC CELLID Cl  ARFCN LEVEL QUAL TYPE SERV

11:49:59 ji -200 6
it Tol 350469.) 444 | 639360 -95 “19/ 5G 2
11:50:06 ji -200 | -19 = 15
15021 CR5GSG| _NIOl (350469. 498 639360, -IS | -14/ | 5G 6
15027 |CR5G5G_ 101350469. 498 | 639360, -119 | -16/ | 5G 2
11:50:29 | CR5GS5G 11468288 534888-. 434 | 52010 -80 Vv 5G 2
115052 ji -200 | -l = 20
TSKI3_CR5GS5G 11468288 534888-. 434 | 5020-85 Vv 5G 20
15133 ji -200 | -l = 54
11:52:27 | CRSGSG 11468288 

In [ ]:
print(os.environ['PATH'])

In [ ]:
tdfx['TXTFFN'] = tdfx.PNGFFN.str.replace(".png",".txt")
tdfx['TXTEXISTS'] = tdfx.TXTFFN.map(lambda xx: os.path.exists(xx))
tdfx['CSVFFN'] = tdfx.PNGFFN.str.replace(".png",".csv")
tdfx['CSVEXISTS'] = tdfx.CSVFFN.map(lambda xx: os.path.exists(xx))
tdfx['PNGDATE'] = tdfx.PNGFFN.map(lambda xx: os.path.basename(xx).replace("Screenshot_","").replace(".png",""))
tdfx = tdfx.sort_values(["PNGFFN"]).reset_index(drop=True)
dumpdf(tdfx)

In [ ]:
print(tdfx.TXTFFN.iloc[0])

In [ ]:
def classifyFile(row):
    fn = row['TXTFFN']
    # print(fn)
    lines = []
    datalst = []
    fclass = "UNKNOWN"
    with open(fn,'r') as f:
        lines = f.readlines()
    lines = [line.rstrip('\n') for line in lines if line != "\n" and line != " "]
    pingfile, kwline = kwInFile(lines,"PING")
    if pingfile:
        fclass = "PING"
        pdest = kwline.split(" ")[0]
        haspstats, pstats = kwInFile(lines,"Ping statistics:",offset=1)
        haststats, tstats = kwInFile(lines,"Time statistics:",offset=1)
        # print(f"pdest={pdest},hasstats={haspstats}, pstats={pstats}, tstats={tstats},kwline={kwline}\n\n{lines}")
        datalst = [pdest,haspstats, pstats, haststats, tstats]
    iperfile, kwline = kwInFile(lines,"bit/sec")
    if iperfile:
        fclass = "IPERF"
        hasidest, kwline = kwInFile(lines," START")
        idest = kwline.split(" ")[0]
        hasircv, ircv = kwInFile(lines,"Receiver summary",offset=1)
        hasisnd, isnd = kwInFile(lines,"Sender summary",offset=1)
        # print(f"idest={idest},hasircv={hasircv}, ircv={ircv}, isnd={isnd}")
        datalst = [idest,hasircv,ircv,hasisnd,isnd]
        # print(datalst)
    cellfile, kwline = kwInFile(lines,"Longitude:")
    if cellfile:
        fclass = "CELL"
        geoloc = kwline
        hascperf, cperf = kwInFile(lines,"SNR:")
        datalst = [geoloc, hascperf, cperf]

    mapfile, kwline = kwInFile(lines,"314-737")
    if mapfile:
        fclass = "MAP"
        hasmperf, mperf = kwInFile(lines,"SNR:")
        hasminfo, minfo = kwInFile(lines,"PCI:")
        datalst = [hasmperf, mperf, hasminfo,minfo]

    speedfile, kwline = kwInFile(lines,"Internet speed test")
    if speedfile:
        fclass = "SPEED"
        hasspeed, speed = kwInFile(lines,"Internet speed test",offset=1)
        datalst = [hasspeed, speed]
    
    return fclass,datalst

def kwInFile(lines,kw,offset=0):
    retval = False
    for ii,line in enumerate(lines):
        if kw in line:
            retval = True
            break
    return retval,lines[ii+offset]
    

tdfx[['FCLASS','DATALST']] = tdfx.apply(classifyFile,axis=1,result_type="expand")
dumpdf(tdfx,head = 20)
# tdfx[:6].apply(classifyFile,axis=1)


In [ ]:
tdfx['DATAFIELD1'] = tdfx.DATALST.map(len).max()

In [ ]:
tdfx[['DATAFIELD1','DATAFIELD2','DATAFIELD3','DATAFIELD4','DATAFIELD5']] = pd.DataFrame(tdfx.DATALST.tolist(),index = tdfx.index)

In [ ]:
dumpdf(tdfx)

In [ ]:
tdfx = drp_lst(tdfx,["DATALST"])
dumpdf(tdfx)

In [ ]:
writejoin(tdfx,".","tmp.csv")

In [ ]:
del tdfx

In [ ]:
tdfy = drp_lst(readjoin(".","tmp-edited.csv"),['Unnamed: 0'])
dumpdf(tdfy)

In [ ]:
# Ping Stats
bdf = (tdfy.FCLASS == "PING") & (tdfy.DATAFIELD2)
tdfy['PINGSTATS'] = tdfy[bdf].DATAFIELD3.map(lambda xx: [xx.split(" ")[0], float(xx.split(" ")[4].strip("%"))])
dumpdf(tdfy[bdf].PINGSTATS,head=len(tdfy))

In [ ]:
# Ping Times
bdf = (tdfy.FCLASS == "PING") & (tdfy.DATAFIELD4)
tdfy['PINGTIMES'] = tdfy[bdf].DATAFIELD5.map(lambda xx: re.sub("\s\s+" , " ", xx.replace("\\","")).split(" ")[1::2])
dumpdf(tdfy[bdf].PINGTIMES,head=len(tdfy))

In [ ]:
# Iperf
def parseIperf(txt):
    try:
        retval = float(re.search(r'\S+$',txt).group(0).rstrip("M"))
    except:
        retval = np.nan
    return retval
bdf = (tdfy.FCLASS == "IPERF") & (tdfy.DATAFIELD4) & (tdfy.DATAFIELD2)
tdfy['IPERFRCV'] = tdfy[bdf][0:].DATAFIELD3.map(parseIperf)
tdfy['IPERFSND'] = tdfy[bdf][0:].DATAFIELD5.map(parseIperf)
dumpdf(tdfy[bdf][['DATAFIELD3','IPERFRCV','DATAFIELD5','IPERFSND']],head=len(tdfy))

In [ ]:
# Cell
bdf = (tdfy.FCLASS == "CELL") & (tdfy.DATAFIELD1.str.len() > 0)
tdfy['GEOLOC'] = tdfy[bdf].DATAFIELD1.map(lambda xx: re.sub("\s\s+" , " ", xx.replace(":","")).split(" ")[1::2])
tdfy['GEOLOC'] = tdfy[bdf].GEOLOC.map(lambda xx: [float(yy) for yy in xx])
type(tdfy[bdf].GEOLOC.iloc[0][0])

def parseCell(txt):
    retval = []
    if txt == '':
        return retval
    ntxt = re.sub("/","",txt)
    try:
        retval = [float(xx) for xx in re.search("^RSRP.*: [-+]?[0-9]+ RSRQ: [-+]?[0-9]+ SNR: [-+]?[0-9]+",ntxt).group(0).split(" ")[1::2]]
        print(f"'{retval}'")
    except:
        retval = []
    return retval
bdf = (tdfy.FCLASS == "CELL") & (tdfy.DATAFIELD2)
tdfy['MAPSIGNAL'] = tdfy[bdf].DATAFIELD3.map(parseCell)
# dumpdf(tdfy[bdf].DATAFIELD3)

In [ ]:
# Map
def parseCellSignal(txt):
    # print(txt)
    retval = []
    if txt == '':
        return retval
    ntxt = re.sub("/","",txt)
    try:
        retval = [float(xx) for xx in re.search("^RSRP.*: [-+]?[0-9]+ RSRQ: [-+]?[0-9]+ SNR: [-+]?[0-9]+",ntxt).group(0).split(" ")[1::2]]
        # print(f"'{retval}'")
    except:
        try:
            retval = [float(xx) for xx in re.search("RSRQ: [-+]?[0-9]+ SNR: [-+]?[0-9]+",ntxt).group(0).split(" ")[1::2]]
            retval.insert(0,np.nan)
            # print(f"'{retval}'")
        except:
            retval = []
    return retval
bdf = (tdfy.FCLASS == "MAP")  & (tdfy.DATAFIELD1)
tdfy['CELLSIGNAL'] = tdfy[bdf].DATAFIELD2.map(parseCellSignal)

def parseCellInfo(txt):
    retval = []
    try:
        retval = re.search("gNB [0-9]+ Cl [0-9]+ PCI [0-9]+",re.sub(":"," ",txt)).group(0).split(" ")[1::2]
    except:
        retval = []
    return retval

bdf = (tdfy.FCLASS == "MAP")  & (tdfy.DATAFIELD3)
tdfy['CELLINFO'] = tdfy[bdf].DATAFIELD4.map(parseCellInfo)

In [ ]:
# Speed Test
bdf = (tdfy.FCLASS == "SPEED")  & (tdfy.DATAFIELD1)
dumpdf(tdfy[bdf].DATAFIELD2,head=20)
tdfy['SPEEDINFO'] = tdfy[bdf].DATAFIELD2.map(lambda xx: [float(yy) for yy in xx.split(" ")])

In [ ]:
writejoin(tdfy.set_index('PNGFFN'),EXPDIR,"tmp-parsed.csv")

### Expand Data

In [ ]:
tdfy = readjoin(EXPDIR,"tmp-parsed.csv")

In [ ]:
print(tdfy.columns)

In [ ]:
keepcol=['GROUP', 'PNGDATE','TEST_UE', 'SCELLID', 'PINGSTATS', 'PINGTIMES',
       'IPERFRCV', 'IPERFSND', 'GEOLOC', 'MAPSIGNAL', 'CELLSIGNAL', 'CELLINFO',
       'SPEEDINFO']
tdfy = tdfy[keepcol]
dumpdf(tdfy)

In [ ]:
tgbx = tdfy.groupby('GROUP')

def collapseGroup(fdf):
    rdf = fdf.ffill().iloc[-1].to_frame().T
    return rdf

tdfz = pd.DataFrame()
for key, groupdf in tgbx:
    tdfz = pd.concat([tdfz,collapseGroup(groupdf)])

dumpdf(tdfz)

In [ ]:
def list2Float(sinlst,lstsize=2):
    outlst = []
    inlst = []
    if isinstance(sinlst,str):
        inlst = [item.strip(" '") for item in sinlst.strip("[]").split(",")]
        # print(inlst)
        for item in inlst:
            try:
                outval = float(item)
            except:
                outval = np.nan
            outlst.append(outval)
    else:
        outlst = [np.nan for ii in range(0,lstsize)]
        # print(type(inlst),inlst)
    print(outlst)
    return outlst

In [ ]:
tdfz['FPINGTIMES'] = tdfz.PINGTIMES.apply(list2Float,lstsize=4)
tdfz[['PTMIN','PTMEAN','PTMAX','PTSD']] = pd.DataFrame(tdfz.FPINGTIMES.tolist(),index = tdfz.index)

tdfz['FPINGSTATS'] = tdfz.PINGSTATS.apply(list2Float,lstsize=2)
tdfz[['PSCNT','PSLOSS']] = pd.DataFrame(tdfz.FPINGSTATS.tolist(),index = tdfz.index)

tdfz['FCELLSIGNAL'] = tdfz.CELLSIGNAL.apply(list2Float,lstsize=3)
tdfz[['RSRP','RSRQ','SNR']] = pd.DataFrame(tdfz.FCELLSIGNAL.tolist(),index = tdfz.index)

tdfz['FCELLINFO'] = tdfz.CELLINFO.apply(list2Float,lstsize=3)
tdfz[['GNB','CID','PCI']] = pd.DataFrame(tdfz.FCELLINFO.tolist(),index = tdfz.index)

tdfz['FSPEEDINFO'] = tdfz.SPEEDINFO.apply(list2Float,lstsize=2)
tdfz[['DOWNLOAD','UPLOAD']] = pd.DataFrame(tdfz.FSPEEDINFO.tolist(),index = tdfz.index)

tdfz['FMAPSIGNAL'] = tdfz.MAPSIGNAL.apply(list2Float,lstsize=3)
tdfz[['MRSRP','MRSRQ','MSNR']] = pd.DataFrame(tdfz.FMAPSIGNAL.tolist(),index = tdfz.index)

tdfz['FGEOLOC'] = tdfz.GEOLOC.apply(list2Float,lstsize=2)
tdfz[['LONGITUDE','LATITUDE']] = pd.DataFrame(tdfz.FGEOLOC.tolist(),index = tdfz.index)

tdfz['RSRP'] = tdfz.apply(lambda row: row.RSRP if not np.isnan(row.RSRP) else row.MRSRP,axis=1)
tdfz['RSRQ'] = tdfz.apply(lambda row: row.RSRQ if not np.isnan(row.RSRQ) else row.MRSRQ,axis=1)
tdfz['SNR'] = tdfz.apply(lambda row: row.SNR if not np.isnan(row.SNR) else row.MSNR,axis=1)

In [ ]:
dumpdf(tdfz.RSRP,head=15)
dumpdf(tdfz.RSRQ,head=15)
dumpdf(tdfz.SNR,head=15)

In [ ]:
keepcol=['GROUP', 'PNGDATE','TEST_UE', 'SCELLID', 'IPERFRCV', 'IPERFSND', 'LONGITUDE','LATITUDE', 
        'PTMIN', 'PTMEAN', 'PTMAX', 'PTSD', 'PSCNT', 'PSLOSS','RSRP', 'RSRQ', 'SNR',
         'GNB', 'CID', 'PCI', 'DOWNLOAD', 'UPLOAD']
tdfz = tdfz[keepcol]

In [ ]:
dumpdf(tdfz,head=15)

In [ ]:
writejoin(tdfz.set_index('PNGDATE'),EXPDIR,"M19-9-10-24-processed.csv")

In [ ]:
df = readjoin(EXPDIR,"M19-9-10-24-processed.csv")
dumpdf(df)


### Mapping

In [ ]:
tdfz['geometry'] = pt2geom(tdfz,latcol='LATITUDE',lngcol='LONGITUDE')

In [ ]:
''' Map Boundaries '''
# USA
# northern=49.382808 # latitude
# southern=24.521208
# eastern=-66.945392 # longitude
# western=-124.736342
fulldict = {
 'northwest':[40.434023, -79.966728],
 'southeast':[40.408245, -79.944540],
}

m19dict = {
    'northwest':[40.419001, -79.955492],
    'southeast':[40.411045, -79.945375]
}

pointdict = m19dict
pointdict['northeast'] = [pointdict['northwest'][0],pointdict['southeast'][1]]
pointdict['southwest'] = [pointdict['southeast'][0],pointdict['northwest'][1]]
tdfb = pd.DataFrame.from_dict(pointdict,orient='index',columns=['latitude','longitude'])
tdfb['geometry'] = pt2geom(tdfb,latcol='latitude',lngcol='longitude')
mapgb = df2gp(tdfb.copy())
del tdfb

In [ ]:
''' Landmarks '''
landmdict = {
    "M19ORU": [40.414848, -79.948185 ,"M19ORU"],
    "PTCORU": [40.432589473606846, -79.96473135648209,"PTCORU"]
}
tdfb = pd.DataFrame.from_dict(landmdict,orient='index',columns=['latitude','longitude','label'])
tdfb['geometry'] = pt2geom(tdfb,latcol='latitude',lngcol='longitude')
lmgb = df2gp(tdfb.copy())
del tdfb
dumpdf(lmgb)

In [ ]:
tgp = df2gp(tdfz.copy()).dropna(subset = ['LATITUDE','LONGITUDE'])
tgp = tgp[tgp.LATITUDE < pointdict['northwest'][0]]
dumpdf(tgp)

In [ ]:
lmgb = lmgb[lmgb.latitude < pointdict['northwest'][0]]
dumpdf(lmgb)

In [ ]:
def gp_plotLabels(fgp2,labelcol="label",ax=None,mapon=True,xoffset=0,yoffset=0, figsize=(30,30), **kwargs):
    '''   GeoPandas DataFrame with 'geometry'containing Points '''
    if mapon: 
        fgp2 = fgp2.to_crs(epsg=3857)
    if ax is None: 
        ax = plt.figure().add_subplot(111)
    xlst,ylst = getPointXY(fgp2)
    # print(f"x={x}, y={y}")
    for x,y,label in zip(xlst,ylst,fgp2[labelcol]):
        ax.text(x+xoffset,y+yoffset,label,**kwargs)
    return ax

def plotMap(fgp,title="NONE",saveon=False,filename="tmp.png",ax=None):
    bbox=dict(facecolor='none', edgecolor='red', boxstyle='round')
    plt.rcParams.update({'font.size': 12})
    # ax = gp_plotPoints(mapgb, ax=ax, mapon=True, alpha=0)
    ax = gp_plotPoints(fgp,ax=ax, mapon=True, title=title, s=20, c='red',figsize=(25,25))
    ax = gp_plotPoints(lmgb,ax=ax, mapon=True,s=20, c='blue')
    bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round',alpha=0.75)
    ax = gp_plotLabels(lmgb,ax=ax, xoffset = 20,mapon=True,c='blue',fontsize=10,bbox=bbox)
    bbox['edgecolor'] = "red"
    ax = gp_plotLabels(fgp,ax=ax,xoffset=5,labelcol='LABEL',mapon=True,c='red',fontsize=9,rotation=8,bbox=bbox)



    if saveon:
        savePlot(ax,filename)
    return ax
try:
    del ax
except:
    pass

tgpx = tgp.copy()
tgpx['LABEL'] = tgpx.apply(lambda xx: f"RSRP: {xx.RSRP} SNR: {xx.SNR} PTMEAN: {xx.PTMEAN} DOWNLOAD: {xx.DOWNLOAD} UPLOAD: {xx.UPLOAD}",axis=1)

ax = plotMap(tgpx,title="Mill 19 Performance Testing 9/10/24", filename="Mill19-Performance-Testing-2024-09-10.png",saveon=False)


### KML File Processing

In [10]:
rxlevfiles = [fn for fn in kmlfiles if "rxlev.kml" in fn]
print(rxlevfiles)

['P:\\My Drive\\CMU-LEL\\Mill19\\Images\\Coverage and Performance\\2024-09-18-Mill19-PTC-Coverage\\LEL-UE1\\314737_2024.09.18_08.47.30\\314737_2024.09.18_08.47.30_rxlev.kml', 'P:\\My Drive\\CMU-LEL\\Mill19\\Images\\Coverage and Performance\\2024-09-18-Mill19-PTC-Coverage\\LEL-UE1\\314737_2024.09.18_13.47.18\\314737_2024.09.18_13.47.18_rxlev.kml', 'P:\\My Drive\\CMU-LEL\\Mill19\\Images\\Coverage and Performance\\2024-09-18-Mill19-PTC-Coverage\\LEL-UE1\\314737_2024.09.18_12.56.45\\314737_2024.09.18_12.56.45_rxlev.kml', 'P:\\My Drive\\CMU-LEL\\Mill19\\Images\\Coverage and Performance\\2024-09-18-Mill19-PTC-Coverage\\LEL-UE1\\314737_2024.09.18_08.45.18\\314737_2024.09.18_08.45.18_rxlev.kml', 'P:\\My Drive\\CMU-LEL\\Mill19\\Images\\Coverage and Performance\\2024-09-18-Mill19-PTC-Coverage\\LEL-UE1\\314737_2024.09.18_12.24.16\\314737_2024.09.18_12.24.16_rxlev.kml', 'P:\\My Drive\\CMU-LEL\\Mill19\\Images\\Coverage and Performance\\2024-09-18-Mill19-PTC-Coverage\\LEL-UE1\\314737_2024.09.18_12.4

In [31]:
print(sys.path)

['C:\\Users\\jimbl\\Git\\LELPerformance\\Network_Measurement_Project\\Jupyter\\Coverage and Performance', 'C:\\Users\\jimbl\\Git\\PyUtils\\lib', 'C:\\Users\\jimbl\\.conda\\envs\\py38\\python38.zip', 'C:\\Users\\jimbl\\.conda\\envs\\py38\\DLLs', 'C:\\Users\\jimbl\\.conda\\envs\\py38\\lib', 'C:\\Users\\jimbl\\.conda\\envs\\py38', '', 'C:\\Users\\jimbl\\.conda\\envs\\py38\\lib\\site-packages', 'C:\\Users\\jimbl\\.conda\\envs\\py38\\lib\\site-packages\\GDAL-3.0.2-py3.8-win-amd64.egg', 'C:\\Users\\jimbl\\.conda\\envs\\py38\\lib\\site-packages\\win32', 'C:\\Users\\jimbl\\.conda\\envs\\py38\\lib\\site-packages\\win32\\lib', 'C:\\Users\\jimbl\\.conda\\envs\\py38\\lib\\site-packages\\Pythonwin']


In [19]:
''' Concatentate Them '''
allfiles = []
for fn in rxlevfiles[:2]:
    with open(fn,"r") as f:
        infile = f.readlines()
        allfiles += infile
        break
print(type(allfiles), allfiles)

<class 'list'> ['<?xml version="1.0" encoding="UTF-8"?><kml xmlns="http://earth.google.com/kml/2.1"><Folder><Placemark><name>-91 5G</name><ExtendedData><Data name="TECHNOLOGY"><value>5G</value></Data><Data name="RSRP"><value>-91 dBm</value></Data><Data name="SPEED"><value>3 km/h</value></Data><Data name="ALTITUDE"><value>195m</value></Data><Data name="TIME"><value>2024.09.18_10.39.42</value></Data></ExtendedData><Style><LabelStyle><scale>0.0</scale> </LabelStyle><IconStyle><color>FFF07800</color><scale>0.3</scale><Icon><href>http://maps.google.com/mapfiles/kml/shapes/shaded_dot.png</href></Icon></IconStyle></Style><Point><coordinates>-79.94860467,40.414949,0</coordinates></Point></Placemark><Placemark><name>-79 5G</name><ExtendedData><Data name="TECHNOLOGY"><value>5G</value></Data><Data name="RSRP"><value>-79 dBm</value></Data><Data name="SPEED"><value>10 km/h</value></Data><Data name="ALTITUDE"><value>200m</value></Data><Data name="TIME"><value>2024.09.18_10.39.43</value></Data></Exte

In [16]:
with open("tmp.kml","w") as f:
    f.writelines(allfiles)


In [28]:
__author__ = 'kristofc'
from xml.etree import ElementTree as eT
import sys

OUTPUT_FILENAME = "MergedOutput.kml"
NAMESPACE = "http://earth.google.com/kml/2.1"


class XMLCombiner(object):
    def __init__(self, filenames):
        assert len(filenames) > 0, 'No filenames!'
        eT.register_namespace('', NAMESPACE)
        # save all the roots, in order, to be processed later
        self.trees = [eT.parse(f) for f in filenames]
        self.roots = [t.getroot() for t in self.trees]

    def combine(self):
        rootcoordinate = self.roots[0].findall(".//{"+NAMESPACE+"}coordinates")
        for r in self.trees[1:]:
            print(r)
            othercoordinate = r.findall(".//{"+NAMESPACE+"}coordinates")
            newcoordinate = str(rootcoordinate[0].text) + str(othercoordinate[0].text)
            filtered = "\n".join([ll.rstrip() for ll in newcoordinate.splitlines() if ll.strip()])
            rootcoordinate[0].text = str(filtered)
            self.trees[0].write(OUTPUT_FILENAME)


# if __name__ == '__main__':
#     if len(sys.argv) > 1:
#         XMLCombiner(sys.argv[1:]).combine()
#     else:
#         print("Supply the files you want to merge on the commandline in the right order.")

In [30]:
XMLCombiner(rxlevfiles[:2]).combine()